In [1]:
import json

In [2]:
trumpPOTUS_members = []
rwb_trump_members = []
woo_members = []
DATA_DIR = 'data/fb_group_member_lists/'
with open(DATA_DIR + 'TrumpPOTUS1.json','r') as f:
    member = json.loads(f.read())
    print(len(member['TrumpPOTUS']))
    member_set = {frozenset(item.items()): item for item in member['TrumpPOTUS']}.values()
    print(len(member_set))
    trumpPOTUS_members = {v['link']: v for v in member_set}
    # for i in member['TrumpPOTUS1']:
    #     arr1.append(i['link'])
    f.close()
    
with open(DATA_DIR + 'RedWhiteandWETHEPEOPLE.json', 'r') as f:
    member = json.loads(f.read())
    print(len(member['Red_White_and_WE_THE_PEOPLE']))
    member_set = {frozenset(item.items()): item for item in member['Red_White_and_WE_THE_PEOPLE']}.values()
    print(len(member_set))
    rwb_trump_members = {v['link']: v for v in member_set}
    # for i in member['TrumpPOTUS1']:
    #     arr1.append(i['link'])
    f.close()
    
with open(DATA_DIR + 'WoothePeople.json', 'r') as f:
    member = json.loads(f.read())
    print(len(member['Woo_the_People']))
    member_set = {frozenset(item.items()): item for item in member['Woo_the_People']}.values()
    print(len(member_set))
    woo_members = {v['link']: v for v in member_set}
    # for i in member['TrumpPOTUS1']:
    #     arr1.append(i['link'])
    f.close()
    

6792
6662
3857
3827
3364
3274


In [3]:
print(woo_members['https://www.facebook.com/woothepeople/'])

{'name': 'Woo the People', 'link': 'https://www.facebook.com/woothepeople/', 'profile_pic': 'https://scontent-lga3-2.xx.fbcdn.net/v/t1.6435-1/61009527_866896013663696_7985116086196502528_n.png?stp=cp0_dst-png_p60x60&_nc_cat=106&ccb=1-5&_nc_sid=1eb0c7&_nc_ohc=QSa2ivwAaCkAX_Ss7Z4&tn=eFsTMwWFOffHNwfJ&_nc_ht=scontent-lga3-2.xx&oh=00_AT8usvwZF8s_f68m9NksGj-G8QK5pQbS59dKQHGQVRahyg&oe=6287F0FE', 'role': 'admin/moderator'}


In [4]:
# Calculate the mutual group members between the three groups
mutual_friends = {'TrumpPOTUS-RWBTrump': [], 'TrumpPOTUS-Woo': [], 'RWBTrump-Woo': [], 'all': []}
mutual_friends['TrumpPOTUS-RWBTrump'] = set(trumpPOTUS_members.keys()) & set(rwb_trump_members.keys())
mutual_friends['TrumpPOTUS-Woo'] = set(trumpPOTUS_members.keys()) & set(woo_members.keys())
mutual_friends['RWBTrump-Woo'] = set(rwb_trump_members.keys()) & set(woo_members.keys())
members_set_list = [set(trumpPOTUS_members.keys()), set(rwb_trump_members.keys()), set(woo_members.keys())]
# unpack the list of sets and pass all of them as arguments into intersection()
mutual_friends['all'] = members_set_list[0].intersection(*members_set_list)

print({type(intersect) for intersect in mutual_friends})
print(len(mutual_friends['TrumpPOTUS-RWBTrump']))
print(len(mutual_friends['TrumpPOTUS-Woo']))
print(len(mutual_friends['RWBTrump-Woo']))
print(len(mutual_friends['all']))
print(len(trumpPOTUS_members))

{<class 'str'>}
5
3
59
1
4792


In [33]:
from pydantic import BaseModel, Extra, HttpUrl
from typing import Optional, Any
import shortuuid


class Node(BaseModel):
    _id: Optional[Any]
    type: Optional[str]
    name: str
    link: str
    
    class Config:
        extra = Extra.ignore
    
    def to_memgraph_format(self):
            return {
                "id": self._id,
                "properties": {k: v for k, v in dict(self).items() if not k.startswith("_")},
                "type": "node"
            }
    

class UserAccount(Node):
    profile_pic: str
    description: Optional[str]
    # def __init__(self, **data):
    #     super().__init__(self, **data)
    #     self.type = "User"
    #     self.photo = data.get("profile_pic")
    #     self.description = data.get("description")
        
class Group(Node):
    type = "Group"
    # def __init(self, **data):
    #     super().__init__(self, **data)
    #     self.type = "Group"

class Relationship(BaseModel):
    _id: Optional[Any]
    source: Optional[Any]
    target: Optional[Any]
    
    class Config:
        extra = Extra.allow
        
    def to_memgraph_format(self):
            return {
                "id": self._id,
                "start": self.source,
                "end": self.target,
                "label": self.__class__.__name__,
                "properties": {k: v for k, v in dict(self).items() if not k.startswith("_")},
                "type": "relationship"
            }
    


In [20]:
nodes_edges_output_list = {"nodes": [], "edges": []}

# nodes_edges_output_list['nodes'].append(Group(id=1, name="PRESIDENT DONALD TRUMP", link="https://www.facebook.com/groups/515283525287467/").dict())
# nodes_edges_output_list['nodes'].append(Group(id=2, name="RED, WHITE, and WE THE PEOPLE back the BLUE & PRESIDENT TRUMP", link="https://www.facebook.com/groups/3838333272856171/").dict())
# nodes_edges_output_list['nodes'].append(Group(id=3, name="Woo the People", link="https://www.facebook.com/groups/355954155643336/").dict())

# Expects input of the form {'group1_name': {'link': group1_link, 'members': group1_members_dict}, 
# 'group2_name': {'link': group2_link, 'members': group2_members_dict}, ...}
# def populate_output_from_dicts(**kwargs):
full_members_dict = {**trumpPOTUS_members, **rwb_trump_members, **woo_members}
for value in full_members_dict.values():
    value["id"] = shortuuid.uuid()
    nodes_edges_output_list["nodes"].append(value)

groups_dict = {
    "https://www.facebook.com/groups/[REDACTED]/": {
        "name": "PRESIDENT DONALD TRUMP", 
        "id": shortuuid.uuid(), 
        "members": trumpPOTUS_members, 
        "link": "https://www.facebook.com/groups/[REDACTED]/",
        "labels": ["Group"]}, 
    "https://www.facebook.com/groups/[REDACTED]/": {
        "name": "RED, WHITE, and WE THE PEOPLE back the BLUE & PRESIDENT TRUMP", 
        "id": shortuuid.uuid(), 
        "members": rwb_trump_members, 
        "link": "https://www.facebook.com/groups/[REDACTED]/",
        "labels": ["Group"]
    },
    "https://www.facebook.com/groups/[REDACTED]/": {
        "name": "Woo the People", 
        "id": shortuuid.uuid(), 
        "members": woo_members, 
        "link": "https://www.facebook.com/groups/[REDACTED]/",
        "labels": ["Group"]}
}



for grp in groups_dict.values():
    source_id = grp["id"]
    nodes_edges_output_list['nodes'].append(grp)
    for k, v in grp["members"].items():
        # Here, rather than simply taking the dictionary item I've been given directly, I will 
        # use the item's key to retrieve the corresponding entry from full_members_dict (where there are no duplicate member entries), 
        # which contains the necessary id property for the member
        member_node_id = full_members_dict[k]["id"]
        nodes_edges_output_list['edges'].append({"source": source_id, "target": member_node_id, "join_date": v.get('joined')})
        
    


In [34]:
memgraph_import_list = []

for node in nodes_edges_output_list['nodes']:
    memgraph_node = None
    if ("UserAccount" in node['labels']):
        memgraph_node = UserAccount.parse_obj(node)
    elif ("Group" in node['labels']):
        memgraph_node = Group.parse_obj(node)
    memgraph_import_list.append(memgraph_node)
    
for edge in nodes_edges_output_list['edges']:
    edge['_id'] = shortuuid.uuid()
    memgraph_edge = Relationship.parse_obj(edge)
    memgraph_import_list.append(memgraph_edge)

In [35]:
print(len(nodes_edges_output_list['nodes']))
print(len(nodes_edges_output_list['edges']))
memgraph_output = list(map(lambda x: x.to_memgraph_format(), memgraph_import_list))

with open('data-temp/network.json', 'w') as f:
    json.dump(nodes_edges_output_list, f, indent=4)
with open('data-temp/memgraph-import.json', 'w') as f:
    json.dump(memgraph_output, f, indent=4)
# with open('nodes.json', 'w') as f:
#     json.dump(nodes_edges_output_list['nodes'], f, indent=4)
# with open('edges.json', 'w') as f:
#     json.dump(nodes_edges_output_list['edges'], f, indent=4)


11829
11892


In [ ]:
# Oops. A nodes-edges list works as an input format for D3, but not for ORA. ORA requires an adjacency matrix in CSV format.
# Let's fix that. 

import numpy as np
import networkx
import pandas as pd 

n_nodes = len(nodes_edges_output_list["nodes"])
A = np.zeros((500,500))
reduced_edges_list = []

for grp in groups_dict.values():
    source_id = grp["id"]
    for index, (k, v) in enumerate(grp["members"].items()):
        # Here, rather than simply taking the dictionary item I've been given directly, I will 
        # use the item's key to retrieve the corresponding entry from full_members_dict (where there are no duplicate member entries), 
        # which contains the necessary id property for the member
        if index <= 1999:
            member_node_id = full_members_dict[k]["id"]
            reduced_edges_list.append({"source": source_id, "target": member_node_id, "join_date": v.get('joined')})
        else:
            break
edges_df = pd.DataFrame(reduced_edges_list)
G = networkx.from_pandas_edgelist(edges_df, edge_attr=True)

reduced_nodes_list = []

for node in G.nodes:
    member_node_entry = nodes_edges_output_list["nodes"][node]
G.nodes = networkx.set_n
# A = networkx.adjacency_matrix(G, weight="join_date")


In [ ]:
networkx.write_weighted_edgelist(G, "data-temp/network_ORA_reduced.csv", delimiter=",")